In [1]:
%matplotlib inline
from clawpack import pyclaw
from clawpack import seismic
from clawpack.clawutil import data
from clawpack.seismic.mappings import Mapping2D
import clawpack.seismic.dtopotools_horiz_okada_and_1d as dtopotools
import numpy as np
from clawpack.visclaw.ianimate import ianimate

num_dim = 2
rundata = data.ClawRunData('amrclaw', num_dim)

In [2]:
!make elasticity_vc_mapped_fault_abl.so > tmp
import elasticity_vc_mapped_fault_abl
!make sloping_fault.so > tmp
import sloping_fault

In [3]:
from numpy import arange,cos,sin,pi,append,linspace
from clawpack.geoclaw.data import LAT2METER

fault = dtopotools.Fault(coordinate_specification='top center')
fault.subfaults = []

width = 50000.0
theta = 0.20
fault_top_center = [0.0,-20000.0]
slip = 1.0
mu = 3e10
rupture_time = 0.0
rise_time = 1.0
nsubfaults = 1 #nsubfaults is statically set in fault_module

longitude0 = fault_top_center[0]/LAT2METER
dlongitude = width*cos(theta)/LAT2METER / nsubfaults
ddepth = width*sin(theta) / nsubfaults
subfault_width = width/nsubfaults

for i in range(nsubfaults):
    subfault = dtopotools.SubFault()
    subfault.mu = mu
    subfault.dip = theta/pi*180.0
    subfault.width = subfault_width
    subfault.depth = -fault_top_center[1] + ddepth*i
    subfault.slip = slip
    subfault.rake = 90
    subfault.strike = 0
    subfault.length = 1000e3
    subfault.longitude = longitude0 + dlongitude*i
    subfault.latitude = 0.
    subfault.coordinate_specification = 'top center'
    subfault.rupture_time = rupture_time
    subfault.rise_time = rise_time

    fault.subfaults.append(subfault)

fault.write('fault.data')

In [4]:
sloping_fault.fault_module.load_fault('fault.data')

In [5]:
fault = dtopotools.Fault()
fault.read('fault.data')

mapping = Mapping2D(fault)
fault_width = mapping.fault_width
fault_depth = mapping.fault_depth
fault_center = mapping.xcenter

In [6]:
clawdata = rundata.clawdata  # initialized when rundata instantiated
probdata = rundata.new_UserData(name='probdata',fname='setprob.data')
probdata.add_param('abl_depth', 10e3, 'depth of absorbing layer')
probdata.add_param('domain_depth', 50e3, 'depth of domain')
probdata.add_param('domain_width', 300e3, 'width of domain')

# ---------------
# Spatial domain:
# ---------------

# Number of space dimensions:
clawdata.num_dim = num_dim

# Number of grid cells:
num_cells_fault = 320 #(5*8*8)
dx = fault_width/num_cells_fault

# determine cell number and set computational boundaries
target_num_cells = np.rint(probdata.domain_width/dx)    # x direction
num_cells_below = np.rint((target_num_cells - num_cells_fault)/2.0)
num_cells_above = target_num_cells - num_cells_below - num_cells_fault
clawdata.lower[0] = fault_center-0.5*fault_width - num_cells_below*dx
clawdata.upper[0] = fault_center+0.5*fault_width + num_cells_above*dx
clawdata.num_cells[0] = int(num_cells_below + num_cells_fault + num_cells_above)

num_cells_above = np.rint(fault_depth/dx) # y direction
dy = fault_depth/num_cells_above
target_num_cells = np.rint(probdata.domain_depth/dy)
num_cells_below = target_num_cells - num_cells_above
clawdata.lower[1] = -fault_depth - num_cells_below*dy
clawdata.upper[1] = 0.0
clawdata.num_cells[1] = int(num_cells_below + num_cells_above)

# add absorbing layer
target_num_cells = np.rint(probdata.abl_depth/dx)
clawdata.lower[0] -= target_num_cells*dx
clawdata.upper[0] += target_num_cells*dx
clawdata.num_cells[0] += 2*int(target_num_cells)
target_num_cells = np.rint(probdata.abl_depth/dy)
clawdata.lower[1] -= target_num_cells*dy
clawdata.num_cells[1] += int(target_num_cells)

# adjust probdata
probdata.domain_width = clawdata.upper[0] - clawdata.lower[0]
probdata.domain_depth = clawdata.upper[1] - clawdata.lower[1]

sloping_fault.ablparam.abldepth = probdata.abl_depth
sloping_fault.ablparam.ablxpos = np.array([clawdata.lower[0]+probdata.abl_depth,
                                           clawdata.upper[0]-probdata.abl_depth])
sloping_fault.ablparam.ablypos = clawdata.lower[1]+probdata.abl_depth

In [7]:
solver = pyclaw.ClawSolver2D(elasticity_vc_mapped_fault_abl)

solver.num_waves = 4
solver.num_eqn = 5
num_aux = 16

x = pyclaw.Dimension(clawdata.lower[0],clawdata.upper[0],clawdata.num_cells[0],name='x')
z = pyclaw.Dimension(clawdata.lower[1],clawdata.upper[1],clawdata.num_cells[1],name='z')

domain = pyclaw.Domain([x,z])
grid = domain.grid
state = pyclaw.State(domain,solver.num_eqn,num_aux)

solver.bc_lower[0] = pyclaw.BC.extrap
solver.bc_upper[0] = pyclaw.BC.extrap
solver.bc_lower[1] = pyclaw.BC.extrap
solver.bc_upper[1] = pyclaw.BC.wall

solver.aux_bc_lower[0] = pyclaw.BC.extrap
solver.aux_bc_upper[0] = pyclaw.BC.extrap
solver.aux_bc_lower[1] = pyclaw.BC.extrap
solver.aux_bc_upper[1] = pyclaw.BC.extrap

state.q[:,:,:] = 0.

mx = grid.x.num_cells
mz = grid.z.num_cells

In [8]:
# Pretend num_ghost = 0 since state.aux doesn't have ghost cells
state.aux[:,:,:] = sloping_fault.setaux(0,mx,mz,grid.x.lower,grid.z.lower,
                           grid.x.delta,grid.z.delta,state.num_aux)

In [9]:
def set_slip(solver,state):
    import copy
    aux = state.aux.copy(order='F')
    sloping_fault.b4step2(0,grid.x.num_cells,grid.z.num_cells,state.q,grid.x.lower,grid.z.lower,
           grid.x.delta,grid.z.delta,state.t,solver.dt,
           aux)
    state.aux[:,:,:] = aux[:,:,:]
solver.before_step = set_slip

In [10]:
claw = pyclaw.Controller()
claw.tfinal = 100
claw.solution = pyclaw.Solution(state,domain)
claw.solver = solver
claw.keep_copy = True
claw.num_output_times = 50

In [ ]:
set_slip(solver,state)

In [ ]:
claw.run()

2017-03-22 16:20:15,218 INFO CLAW: Solution 0 computed for time t=0.000000


In [ ]:
#ianimate(claw)